In [1]:
!pip3 install nltk googledrivedownloader gensim
import gensim.models as g
import logging
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1shL8-LgL4HaaWSYCb2K09PlyomGFREEB', dest_path='./sinopses.txt', unzip=False) # pesa 3GB

Aqui definimos as varíaveis que serão usadas como parâmetro no nosso treinamento.

In [0]:
#doc2vec parameters
vector_size = 300
window_size = 15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 2
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 2

O córpus que utilizaremos é um de sinopses de filmes com aproximadamente 10496 sinopses.

In [0]:
#input corpus
train_corpus = "sinopses.txt" #train_docs

Faremos a remoção de _stopwords_ para eliminar ruído dos dados.

In [5]:
#stopwords
pt_stop = stopwords.words('portuguese')
pt_stop.extend(['para','que'])
print(pt_stop)

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos

Esse método faz a leitura do córpus enquanto remove as _stopwords_.

* **TaggedDocument**: Adiciona uma _label_ para cada documento (sinopse). Neste caso, estamos adicionando o contador como _label_. Normalmente se usa um id inteiro único.

In [0]:
def read_corpus(fname, tokens_only=False): 
    with open(fname) as f: #
        read_data = f.readlines()
        for i, line in enumerate(read_data):
            t_corpus = line.split(" | ")
            if(len(t_corpus)==2):
                #removing stopwords
                words = t_corpus[1].split()
                filtered_words = [word for word in words if word not in pt_stop]
                yield g.doc2vec.TaggedDocument(filtered_words, [i])

In [0]:
train_docs = list(read_corpus(train_corpus))

In [8]:
train_docs[:5]

[TaggedDocument(words=['As', 'coisas', 'mal', 'Inteligência', 'Britânica,', 'pois', 'Smersh', 'começara', 'sabotar', 'estabilidade', 'global:', 'nada', 'menos', 'onze', 'agentes', 'abatidos', 'e,', 'piorar', 'coisas,', 'maior', 'agente', 'secreto,', '007,', 'desfrutando', 'aposentadoria.', 'Sir', 'James', 'Bond,', 'primeiro', '007,', 'é', 'convencido', 'alguns', 'chefes', 'agências', 'espionagem', 'combater', 'inimigo', 'comum.', 'Essa', 'versão', '"Cassino', 'Royale"', 'é', 'versão', 'oficial', 'filmes', '007,', 'pois', 'rodado', 'outra', 'equipe,', 'estúdios,', 'padrões', 'contratos.', 'É', 'produção', 'anglo-americana', '1967,', 'gênero', 'comédia', 'espionagem.'], tags=[0]),
 TaggedDocument(words=['A', 'ação', 'é', 'eletrizante', 'ininterrupta,', 'pois', 'agente', '007', '(Sean', 'Connery)', 'vai', 'além', 'dever', 'ofício', 'profundezas', 'oceano', 'encontrar', 'perigoso', 'criminoso', 'ameaçando', 'milhões', 'pessoas', 'através', 'chantagem', 'destruir', 'mundo', 'meio', 'holocau

In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Parâmetros

Na próxima célula de código, definimos os seguintes parâmetros:
* **size**: dimensionalidade dos vetores.

* **window**: é a quantidade de palavras anteriores e posteriores à palavra alvo.

* **min_count**: ignore as palavras com frequência total inferior a **min_count**.

* **sample**: limiar para configurar quais palavras de maior frequência são aleatoriamente reduzidas; O padrão é 1e-3, o intervalo útil é (0, 1e-5).

* **workers**: parâmetro que indica quantos cores da máquina serão utilizados para o treinamento.

* **hs**: se 1, softmax hierárquico será usado para o treinamento do modelo. Se definido como 0 (padrão), e existir amostragem negativa, esse recurso será utilizado.

* **dm**: define o algoritmo de treinamento. Por padrão, o DBOW é usado (dm = 0). O outro é o DMPV (dm = 1).

* **negative**: se > 0, será utilizada amostragem negativa. O valor indica quantas "palavras de ruído" devem ser consideradas (normalmente entre 5 a 20). Se **negative** configurado para 0, não é utilizada a amostragem negativa.

* **dbow_words**: se 1, skip-gram é usado para gerar os vetores de palavras simultaneamente com DBOW; O padrão é 0. Essa funcionalidade aumenta o conjunto de dados ao adicionar os vetores de palavras junto aos de documento. o treinamento ficará mais lento.

* **dm_concat**: se 1, usa a concatenação de vetores de contexto em vez da soma/média; O padrão é 0 (desativado).

* **iter**: número de iterações (épocas) sobre o córpus. O padrão é 5.





In [10]:
model = g.doc2vec.Doc2Vec(size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:566: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-10-15 18:31:42,562 : INFO : using concatenative 9300-dimensional layer1


* **build_vocab**: Método que constrói um dicionário de palavras distintas presentes no córpus.

In [11]:
model.build_vocab(train_docs)

2018-10-15 18:31:43,769 : INFO : collecting all words and their counts
2018-10-15 18:31:43,771 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-10-15 18:31:43,914 : INFO : collected 62995 word types and 7771 unique tags from a corpus of 7529 examples and 296335 words
2018-10-15 18:31:43,915 : INFO : Loading a fresh vocabulary
2018-10-15 18:31:44,174 : INFO : effective_min_count=1 retains 62995 unique words (100% of original 62995, drops 0)
2018-10-15 18:31:44,175 : INFO : effective_min_count=1 leaves 296335 word corpus (100% of original 296335, drops 0)
2018-10-15 18:31:44,380 : INFO : deleting the raw counts dictionary of 62995 items
2018-10-15 18:31:44,383 : INFO : sample=1e-05 downsamples 5677 most-common words
2018-10-15 18:31:44,384 : INFO : downsampling leaves estimated 164586 word corpus (55.5% of prior 296335)
2018-10-15 18:31:44,683 : INFO : estimated required memory for 62995 words and 300 dimensions: 2459830700 bytes
2018-10-15 18:31:44,68

In [12]:
# Aqui ocorre o treinamento do nosso modelo.
%time model.train(train_docs, total_examples=model.corpus_count, epochs=model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.
2018-10-15 18:31:49,126 : INFO : training model with 1 workers on 62996 vocabulary and 9300 features, using sg=0 hs=0 sample=1e-05 negative=5 window=15
2018-10-15 18:31:50,300 : INFO : EPOCH 1 - PROGRESS: at 10.09% examples, 15004 words/s, in_qsize 1, out_qsize 0
2018-10-15 18:31:51,466 : INFO : EPOCH 1 - PROGRESS: at 20.11% examples, 14898 words/s, in_qsize 1, out_qsize 0
2018-10-15 18:31:52,642 : INFO : EPOCH 1 - PROGRESS: at 30.32% examples, 14814 words/s, in_qsize 1, out_qsize 0
2018-10-15 18:31:53,985 : INFO : EPOCH 1 - PROGRESS: at 44.34% examples, 15482 words/s, in_qsize 1, out_qsize 0
2018-10-15 18:31:55,265 : INFO : EPOCH 1 - PROGRESS: at 58.67% examples, 16041 words/s, in_qsize 1, out_qsize 0
2018-10-15 18:31:56,551 : INFO : EPOCH 1 - PROGRESS: at 71.64% ex

CPU times: user 19.5 s, sys: 25.2 ms, total: 19.5 s
Wall time: 19.7 s


In [14]:
# Salva o modelo no disco
model.save("doc2vecmodel.bin")

2018-10-15 18:32:23,630 : INFO : saving Doc2Vec object under doc2vecmodel.bin, separately None
2018-10-15 18:32:23,635 : INFO : storing np array 'syn1neg' to doc2vecmodel.bin.trainables.syn1neg.npy
2018-10-15 18:33:33,381 : INFO : storing np array 'vectors' to doc2vecmodel.bin.wv.vectors.npy
2018-10-15 18:33:34,223 : INFO : saved doc2vecmodel.bin


In [0]:
import codecs
import numpy as np

In [16]:
# Aqui faremos o carregamento do modelo treinado.

m = g.Doc2Vec.load("doc2vecmodel.bin")
print(m.docvecs.count)

2018-10-15 18:33:36,621 : INFO : loading Doc2Vec object from doc2vecmodel.bin
2018-10-15 18:33:36,975 : INFO : loading vocabulary recursively from doc2vecmodel.bin.vocabulary.* with mmap=None
2018-10-15 18:33:36,977 : INFO : loading trainables recursively from doc2vecmodel.bin.trainables.* with mmap=None
2018-10-15 18:33:36,978 : INFO : loading syn1neg from doc2vecmodel.bin.trainables.syn1neg.npy with mmap=None
2018-10-15 18:35:24,432 : INFO : loading wv recursively from doc2vecmodel.bin.wv.* with mmap=None
2018-10-15 18:35:24,438 : INFO : loading vectors from doc2vecmodel.bin.wv.vectors.npy with mmap=None
2018-10-15 18:35:27,792 : INFO : loading docvecs recursively from doc2vecmodel.bin.docvecs.* with mmap=None
2018-10-15 18:35:27,793 : INFO : loaded doc2vecmodel.bin


7771


In [17]:
# Tokenizaremos cada sinopse e adicionaremos as palavras em um vetor que será usado para a inferência.
alldocs = []
cont = 0
with open(train_corpus) as alldata: #, encoding='utf-8'
    for line_no, line in enumerate(alldata):
        words = g.utils.to_unicode(line).split()
        alldocs.append(words)
        print(words)
        cont+=1

In [20]:
doc_id = np.random.randint(m.docvecs.count)  # pick random doc, re-run cell for more examples
#print(m.docvecs.count)
sims = m.docvecs.most_similar(doc_id, topn=m.docvecs.count)  # get *all* similar documents m.docvecs.count
print(u'TARGET (%d): %s\n' % (doc_id, ' '.join(alldocs[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % m)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: %s\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]])))

TARGET (4311): Mrs. Soffel - Um Amor Proibido(1984) | Pittsburgh, 1901. Peter Soffel (Edward Herrmann) é o diretor de uma prisão de segurança máxima e Kate Soffel (Diane Keaton), sua esposa, apesar de ter uma saúde frágil freqüentemente lê a Bíblia em voz alta para os internos. Durante sua ronda habitual Kate fica sabendo dos Irmãos Biddle, Ed (Mel Gibson) e Jack (Matthew Modine), que foram condenados à morte por assassinato e roubo. Ed tem boa aparência, é inteligente e charmoso, fazendo Kate ficar impressionada. Gradativamente, ela se apaixona por Ed e ajuda os dois irmãos a escaparem da prisão, fugindo com eles.

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/c,d300,n5,w15,s1e-05):

MOST (4487, 0.21790188550949097): Noivas | Liv (Kate Hudson) e Emma (Anne Hathaway) são grandes amigas e planejaram seus respectivos casamentos nos mínimos detalhes desde crianças, quando brincavam juntas cantando "Lá vem a noiva" em seus sótãos. Sempre souberam, sem sombra de dúvida, que se casariam no lo

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
